In [1]:
import argparse
import json
import os
import pathlib
import pickle as pkl
import shutil
import sys
import time
import warnings
from collections import defaultdict
from typing import Dict, List, Optional

import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

### schema
import uproot
from coffea import nanoevents, processor
from coffea.analysis_tools import PackedSelection, Weights
from coffea.nanoevents import BaseSchema, NanoAODSchema, NanoEventsFactory
from coffea.nanoevents.methods import candidate, vector

import mplhep as hep

plt.style.use(hep.style.CMS)

### awkward 1.10.0
sys.path.append("../")

nanoevents.PFNanoAODSchema.mixins["PFCands"] = "PFCand"
nanoevents.PFNanoAODSchema.mixins["SV"] = "PFCand"

warnings.filterwarnings("ignore", message="Found duplicate branch ")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="Missing cross-reference index ")
warnings.filterwarnings("ignore", message="divide by zero encountered in log")
np.seterr(invalid="ignore")

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
%load_ext autoreload
%autoreload 2

In [50]:
! ls ../datafiles/ntuples/*

../datafiles/ntuples/GluGluHToWW_Pt-200ToInf_M-125:
test  train

../datafiles/ntuples/QCD_Pt_300to470:
test  train

../datafiles/ntuples/QCD_Pt_470to600:
test  train

../datafiles/ntuples/QCD_Pt_600to800:
test  train

../datafiles/ntuples/QCD_Pt_800to1000:
test  train

../datafiles/ntuples/TTToSemiLeptonic:
test  train

../datafiles/ntuples/WJetsToLNu_HT-200To400:
test  train

../datafiles/ntuples/WJetsToLNu_HT-400To600:
test  train

../datafiles/ntuples/WJetsToLNu_HT-600To800:
test  train

../datafiles/ntuples/WJetsToLNu_HT-800To1200:
test  train


# Higgs file

In [51]:
events = uproot.open("../datafiles/ntuples/GluGluHToWW_Pt-200ToInf_M-125/train/out.root")["Events"]
events.keys()

['fj_eta',
 'fj_phi',
 'fj_mass',
 'fj_pt',
 'fj_msoftdrop',
 'matched_mask',
 'fj_genjetmass',
 'fj_genRes_pt',
 'fj_genRes_eta',
 'fj_genRes_phi',
 'fj_genRes_mass',
 'fj_nprongs',
 'fj_ncquarks',
 'fj_lepinprongs',
 'fj_nquarks',
 'fj_H_VV_4q',
 'fj_H_VV_elenuqq',
 'fj_H_VV_munuqq',
 'fj_H_VV_leptauelvqq',
 'fj_H_VV_leptaumuvqq',
 'fj_H_VV_hadtauvqq',
 'fj_QCDb',
 'fj_QCDbb',
 'fj_QCDc',
 'fj_QCDcc',
 'fj_QCDothers',
 'fj_V_2q',
 'fj_V_elenu',
 'fj_V_munu',
 'fj_V_taunu',
 'fj_Top_nquarksnob',
 'fj_Top_nbquarks',
 'fj_Top_ncquarks',
 'fj_Top_nleptons',
 'fj_Top_nele',
 'fj_Top_nmu',
 'fj_Top_ntau',
 'fj_Top_taudecay',
 'met_relpt',
 'met_relphi',
 'lep_dR_fj',
 'lep_pt',
 'lep_pt_ratio',
 'fj_ParT_inclusive_score',
 'fj_ParT_mass',
 'fj_ParT_hidNeuron000',
 'fj_ParT_hidNeuron001',
 'fj_ParT_hidNeuron002',
 'fj_ParT_hidNeuron003',
 'fj_ParT_hidNeuron004',
 'fj_ParT_hidNeuron005',
 'fj_ParT_hidNeuron006',
 'fj_ParT_hidNeuron007',
 'fj_ParT_hidNeuron008',
 'fj_ParT_hidNeuron009',
 'fj_

In [52]:
events["fj_genRes_mass"].array()

<Array [125, 125, 125, 125, ... 125, 125, 125] type='98507 * float64'>

In [9]:
events["fj_H_VV_elenuqq"].array() + events["fj_H_VV_munuqq"].array()

<Array [0, 1, 0, 0, 0, 0, ... 0, 0, 1, 0, 1, 1] type='98507 * int64'>

In [10]:
events["fj_lepinprongs"].array()

<Array [0, 1, 0, 0, 0, 0, ... 0, 0, 1, 0, 0, 0] type='98507 * int64'>

In [12]:
(events["fj_lepinprongs"].array()==1) & (events["fj_nquarks"].array()==2)

<Array [False, True, False, ... False, False] type='98507 * bool'>

In [13]:
matched_higgs = ( (events["fj_genRes_mass"].array()==125) & (events["matched_mask"].array()) )
ak.sum(matched_higgs)

50890

In [14]:
unmatched_higgs = ( (events["fj_genRes_mass"].array()==125) & ~(events["matched_mask"].array()) )
ak.sum(unmatched_higgs)

47617

In [15]:
a = ( (events["fj_genRes_mass"].array()==125) & (events["matched_mask"].array() & (events["fj_lepinprongs"].array()==1) & (f["Events"]["fj_nquarks"].array()==2)) )
ak.sum(a)

15573

In [17]:
# matched Higgs: target mass = 125GeV
one_lep_inprongs = events["fj_lepinprongs"].array()==1
two_quarks = events["fj_nquarks"].array()==2

events["matched_mask"].array() & one_lep_inprongs & two_quarks

<Array [False, True, False, ... False, False] type='98507 * bool'>

In [190]:
(events["matched_mask"].array() & one_lep_inprongs & two_quarks).to_numpy().sum()

1258

In [18]:
ak.sum((events["fj_V_2q"].array() + events["fj_V_elenu"].array() + events["fj_V_munu"].array() + events["fj_V_taunu"].array())==0)

98507

# QCD file

In [101]:
events = uproot.open("../datafiles/ntuples/QCD_Pt_300to470/train/out.root")["Events"]
events.keys()

['fj_eta',
 'fj_phi',
 'fj_mass',
 'fj_pt',
 'fj_msoftdrop',
 'matched_mask',
 'fj_genjetmass',
 'fj_genRes_pt',
 'fj_genRes_eta',
 'fj_genRes_phi',
 'fj_genRes_mass',
 'fj_nprongs',
 'fj_ncquarks',
 'fj_lepinprongs',
 'fj_nquarks',
 'fj_H_VV_4q',
 'fj_H_VV_elenuqq',
 'fj_H_VV_munuqq',
 'fj_H_VV_leptauelvqq',
 'fj_H_VV_leptaumuvqq',
 'fj_H_VV_hadtauvqq',
 'fj_QCDb',
 'fj_QCDbb',
 'fj_QCDc',
 'fj_QCDcc',
 'fj_QCDothers',
 'fj_V_2q',
 'fj_V_elenu',
 'fj_V_munu',
 'fj_V_taunu',
 'fj_Top_nquarksnob',
 'fj_Top_nbquarks',
 'fj_Top_ncquarks',
 'fj_Top_nleptons',
 'fj_Top_nele',
 'fj_Top_nmu',
 'fj_Top_ntau',
 'fj_Top_taudecay',
 'met_relpt',
 'met_relphi',
 'lep_dR_fj',
 'lep_pt',
 'lep_pt_ratio',
 'fj_ParT_inclusive_score',
 'fj_ParT_mass',
 'fj_ParT_hidNeuron000',
 'fj_ParT_hidNeuron001',
 'fj_ParT_hidNeuron002',
 'fj_ParT_hidNeuron003',
 'fj_ParT_hidNeuron004',
 'fj_ParT_hidNeuron005',
 'fj_ParT_hidNeuron006',
 'fj_ParT_hidNeuron007',
 'fj_ParT_hidNeuron008',
 'fj_ParT_hidNeuron009',
 'fj_

In [102]:
events["fj_genRes_mass"].array()

<Array [0, 0, 0, 0, 0, 0, ... 0, 0, 0, 0, 0, 0] type='38434 * float64'>

In [57]:
events["fj_QCDb"].array() + events["fj_QCDbb"].array() + events["fj_QCDc"].array() + events["fj_QCDcc"].array() + events["fj_QCDothers"].array()

<Array [1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1] type='38434 * int64'>

# WJets file

In [82]:
events = uproot.open("../datafiles/ntuples/WJetsToLNu_HT-200To400/train/out.root")["Events"]
events.keys()

['fj_eta',
 'fj_phi',
 'fj_mass',
 'fj_pt',
 'fj_msoftdrop',
 'matched_mask',
 'fj_genjetmass',
 'fj_genRes_pt',
 'fj_genRes_eta',
 'fj_genRes_phi',
 'fj_genRes_mass',
 'fj_nprongs',
 'fj_ncquarks',
 'fj_lepinprongs',
 'fj_nquarks',
 'fj_H_VV_4q',
 'fj_H_VV_elenuqq',
 'fj_H_VV_munuqq',
 'fj_H_VV_leptauelvqq',
 'fj_H_VV_leptaumuvqq',
 'fj_H_VV_hadtauvqq',
 'fj_QCDb',
 'fj_QCDbb',
 'fj_QCDc',
 'fj_QCDcc',
 'fj_QCDothers',
 'fj_V_2q',
 'fj_V_elenu',
 'fj_V_munu',
 'fj_V_taunu',
 'fj_Top_nquarksnob',
 'fj_Top_nbquarks',
 'fj_Top_ncquarks',
 'fj_Top_nleptons',
 'fj_Top_nele',
 'fj_Top_nmu',
 'fj_Top_ntau',
 'fj_Top_taudecay',
 'met_relpt',
 'met_relphi',
 'lep_dR_fj',
 'lep_pt',
 'lep_pt_ratio',
 'fj_ParT_inclusive_score',
 'fj_ParT_mass',
 'fj_ParT_hidNeuron000',
 'fj_ParT_hidNeuron001',
 'fj_ParT_hidNeuron002',
 'fj_ParT_hidNeuron003',
 'fj_ParT_hidNeuron004',
 'fj_ParT_hidNeuron005',
 'fj_ParT_hidNeuron006',
 'fj_ParT_hidNeuron007',
 'fj_ParT_hidNeuron008',
 'fj_ParT_hidNeuron009',
 'fj_

In [83]:
events["fj_genRes_mass"].array()

<Array [0, 0, 0, 0, 0, 0, ... 0, 0, 0, 0, 0, 0] type='33465 * float64'>

In [84]:
# all wjets events belong to one of the decays

for decay in ["fj_V_2q", "fj_V_elenu", "fj_V_munu", "fj_V_taunu"]:
    print(f"{decay}: {ak.sum(events[decay].array())}")

ak.sum((events["fj_V_2q"].array() + events["fj_V_elenu"].array() + events["fj_V_munu"].array() + events["fj_V_taunu"].array())==0)

fj_V_2q: 0
fj_V_elenu: 13244
fj_V_munu: 14485
fj_V_taunu: 5736


0

In [85]:
# merged Ws: target mass = 80GeV
lep_in_W = (events["fj_V_munu"].array()>0) | (events["fj_V_elenu"].array()>0)

(events["matched_mask"].array() & lep_in_W).to_numpy().sum()

10032

In [86]:
# unmatched Ws: target mass = fj_genjetmass
(~events["matched_mask"].array()).to_numpy().sum()

23043

# Top file

In [87]:
events = uproot.open("../datafiles/ntuples/TTToSemiLeptonic/train/out.root")["Events"]
events.keys()

['fj_eta',
 'fj_phi',
 'fj_mass',
 'fj_pt',
 'fj_msoftdrop',
 'matched_mask',
 'fj_genjetmass',
 'fj_genRes_pt',
 'fj_genRes_eta',
 'fj_genRes_phi',
 'fj_genRes_mass',
 'fj_nprongs',
 'fj_ncquarks',
 'fj_lepinprongs',
 'fj_nquarks',
 'fj_H_VV_4q',
 'fj_H_VV_elenuqq',
 'fj_H_VV_munuqq',
 'fj_H_VV_leptauelvqq',
 'fj_H_VV_leptaumuvqq',
 'fj_H_VV_hadtauvqq',
 'fj_QCDb',
 'fj_QCDbb',
 'fj_QCDc',
 'fj_QCDcc',
 'fj_QCDothers',
 'fj_V_2q',
 'fj_V_elenu',
 'fj_V_munu',
 'fj_V_taunu',
 'fj_Top_nquarksnob',
 'fj_Top_nbquarks',
 'fj_Top_ncquarks',
 'fj_Top_nleptons',
 'fj_Top_nele',
 'fj_Top_nmu',
 'fj_Top_ntau',
 'fj_Top_taudecay',
 'met_relpt',
 'met_relphi',
 'lep_dR_fj',
 'lep_pt',
 'lep_pt_ratio',
 'fj_ParT_inclusive_score',
 'fj_ParT_mass',
 'fj_ParT_hidNeuron000',
 'fj_ParT_hidNeuron001',
 'fj_ParT_hidNeuron002',
 'fj_ParT_hidNeuron003',
 'fj_ParT_hidNeuron004',
 'fj_ParT_hidNeuron005',
 'fj_ParT_hidNeuron006',
 'fj_ParT_hidNeuron007',
 'fj_ParT_hidNeuron008',
 'fj_ParT_hidNeuron009',
 'fj_

In [88]:
events["fj_genRes_mass"].array()

<Array [0, 0, 0, 0, 0, 0, ... 0, 0, 0, 0, 0, 0] type='219138 * float64'>

In [89]:
# fully merged Top: target mass = 175GeV
lep_in_jet = events["fj_Top_nleptons"].array()>0
bquark_in_jet = events["fj_Top_nbquarks"].array()>0

(events["matched_mask"].array() & lep_in_jet & bquark_in_jet).to_numpy().sum()

50146

In [90]:
# W merged Top: target mass = 80GeV
(events["matched_mask"].array() & lep_in_jet & ~bquark_in_jet).to_numpy().sum()

23300

In [91]:
# unmatched Top: target mass = fj_genjetmass
(~events["matched_mask"].array()).to_numpy().sum()

48306

# Check Number of events in ntuples

In [95]:
samples = [
    "GluGluHToWW_Pt-200ToInf_M-125",
    "TTToSemiLeptonic",
    "WJetsToLNu_HT-200To400",
    "WJetsToLNu_HT-400To600",
    "WJetsToLNu_HT-600To800",
    "WJetsToLNu_HT-800To1200",
    "QCD_Pt_300to470",
    "QCD_Pt_470to600",
    "QCD_Pt_600to800",
    "QCD_Pt_800to1000"
]

In [97]:
OUTPATH = "../datafiles/ntuples/"

num_events = {}
for sample in samples:
    
    num = 0
    for file in os.listdir(f"{OUTPATH}/{sample}/train/"):
        if "root" not in file:
            continue        

        events = uproot.open(f"{OUTPATH}/{sample}/train/{file}")["Events"]

        num += events.num_entries
    
    print(sample, "train", num)
     
    num = 0        
    for file in os.listdir(f"{OUTPATH}/{sample}/test/"):
        if "root" not in file:
            continue
        events = uproot.open(f"{OUTPATH}/{sample}/test/{file}")["Events"]
        
        num += events.num_entries

    print(sample, "test", num)
    print("--------------------------")

GluGluHToWW_Pt-200ToInf_M-125 train 98507
GluGluHToWW_Pt-200ToInf_M-125 test 74132
--------------------------
TTToSemiLeptonic train 219138
TTToSemiLeptonic test 219176
--------------------------
WJetsToLNu_HT-200To400 train 33465
WJetsToLNu_HT-200To400 test 33733
--------------------------
WJetsToLNu_HT-400To600 train 55833
WJetsToLNu_HT-400To600 test 57533
--------------------------
WJetsToLNu_HT-600To800 train 58566
WJetsToLNu_HT-600To800 test 62140
--------------------------
WJetsToLNu_HT-800To1200 train 55795
WJetsToLNu_HT-800To1200 test 64846
--------------------------
QCD_Pt_300to470 train 38434
QCD_Pt_300to470 test 38048
--------------------------
QCD_Pt_470to600 train 41663
QCD_Pt_470to600 test 41486
--------------------------
QCD_Pt_600to800 train 42574
QCD_Pt_600to800 test 50102
--------------------------
QCD_Pt_800to1000 train 50103
QCD_Pt_800to1000 test 50455
--------------------------


# Check NaNs in ntuples

In [100]:
OUTPATH = "../datafiles/mass_regression_ntuples_reduced/"
# OUTPATH = "../datafiles/ntuples/"

for sample in samples:
    print(sample)
    
    for file in os.listdir(f"{OUTPATH}/{sample}/train/"):
        if "root" not in file:
            continue        

        events = uproot.open(f"{OUTPATH}/{sample}/train/{file}")["Events"]
        
        for var in events.keys():
            nans = (np.isnan(events[var].array().to_numpy())).sum()
            if nans != 0:
                print(file, nans, f"nan {var} values")
        
     
    for file in os.listdir(f"{OUTPATH}/{sample}/test/"):
        if "root" not in file:
            continue
        events = uproot.open(f"{OUTPATH}/{sample}/test/{file}")["Events"]
        
        for var in events.keys():
            nans = (np.isnan(events[var].array().to_numpy())).sum()
            if nans != 0:
                print(file, nans, f"nan {var} values")
                
    print("--------------------------")

GluGluHToWW_Pt-200ToInf_M-125
0-18.root 1 nan fj_genjetmass values
--------------------------
TTToSemiLeptonic
50-80.root 2 nan fj_genjetmass values
--------------------------
WJetsToLNu_HT-200To400
--------------------------
WJetsToLNu_HT-400To600
12-24.root 1 nan fj_genjetmass values
--------------------------
WJetsToLNu_HT-600To800
0-12.root 1 nan fj_genjetmass values
12-24.root 4 nan fj_genjetmass values
--------------------------
WJetsToLNu_HT-800To1200
12-24.root 3 nan fj_genjetmass values
--------------------------
QCD_Pt_300to470
--------------------------
QCD_Pt_470to600
--------------------------
QCD_Pt_600to800
--------------------------
QCD_Pt_800to1000
--------------------------
